<a href="https://www.kaggle.com/code/edgarabasov/notebookef60b8444c?scriptVersionId=238001136" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score


In [8]:
FOLDER_PATH = '/kaggle/input/star-observations-dataset'
FILE_NAME = "Star_Data_2025.04.16_04.44.28.csv"
FULL_PATH = FOLDER_PATH + '/' + FILE_NAME

In [9]:
df = pd.read_csv(FULL_PATH)

In [10]:
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,star_id,region,ra,dec,starthjd,endhjd,bmag,berr,vmag,verr,rmag,rerr,imag,ierr,lcfil,npts
0,NGC2301_Center_2,NGC2301,103.0431,0.4158,2453047.76,2453058.94,11.139,0.002,NaN,NaN,11.083,0.003,NaN,NaN,R,167
1,NGC2301_Center_3,NGC2301,102.9154,0.4254,2453047.76,2453058.94,12.193,0.001,NaN,NaN,11.869,0.005,NaN,NaN,R,167
2,NGC2301_Center_4,NGC2301,102.9629,0.4276,2453047.76,2453058.94,11.976,0.001,NaN,NaN,11.460,0.003,NaN,NaN,R,167
3,NGC2301_Center_5,NGC2301,103.0485,0.4260,2453047.76,2453058.94,11.065,0.002,NaN,NaN,10.992,0.003,NaN,NaN,R,167
4,NGC2301_Center_6,NGC2301,102.9760,0.4326,2453047.76,2453058.94,11.914,0.002,NaN,NaN,11.668,0.005,NaN,NaN,R,167


In [11]:
df.shape

(138935, 16)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138935 entries, 0 to 138934
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   star_id   138935 non-null  object 
 1   region    138935 non-null  object 
 2   ra        138935 non-null  float64
 3   dec       138935 non-null  float64
 4   starthjd  138935 non-null  float64
 5   endhjd    138935 non-null  float64
 6   bmag      3961 non-null    float64
 7   berr      3961 non-null    float64
 8   vmag      127364 non-null  float64
 9   verr      127364 non-null  float64
 10  rmag      3961 non-null    float64
 11  rerr      3961 non-null    float64
 12  imag      131440 non-null  float64
 13  ierr      131440 non-null  float64
 14  lcfil     138935 non-null  object 
 15  npts      138935 non-null  int64  
dtypes: float64(12), int64(1), object(3)
memory usage: 17.0+ MB


In [13]:
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

In [14]:
df[numeric_columns].describe()

,ra,dec,starthjd,endhjd,bmag,berr,vmag,verr,rmag,rerr,imag,ierr,npts
count,138935.000000,138935.000000,1.389350e+05,1.389350e+05,3961.000000,3961.000000,127364.000000,127364.000000,3961.000000,3961.000000,131440.000000,131440.000000,138935.00000
mean,207.216280,-21.339073,2.450392e+06,2.451170e+06,19.193779,0.008489,20.670804,0.091144,17.331396,0.003215,19.454649,0.068939,88.21410
std,51.492794,21.465881,5.047825e+02,3.699247e+02,2.131007,0.006817,1.523555,0.097095,1.759611,0.002474,1.315436,0.062515,40.16088
min,102.750100,-46.611600,2.449898e+06,2.449903e+06,10.010000,0.001000,15.505000,0.010000,9.844000,0.001000,14.155000,0.010000,8.00000
25%,154.404500,-46.395000,2.450200e+06,2.450941e+06,18.198000,0.005000,19.596000,0.023000,16.503000,0.002000,18.595000,0.023000,51.00000
50%,251.736100,-4.165700,2.450245e+06,2.451053e+06,19.729000,0.007000,20.792000,0.053000,17.788000,0.003000,19.602000,0.047000,92.00000
75%,254.211500,-2.097200,2.450567e+06,2.451319e+06,20.781000,0.010000,21.915000,0.131000,18.655000,0.004000,20.481000,0.096000,126.00000
max,254.486800,0.690800,2.453048e+06,2.453059e+06,22.449000,0.105000,25.097000,2.862000,19.478000,0.065000,23.077000,1.223000,175.00000


In [15]:
len(numeric_columns)

13

In [16]:
df[numeric_columns].isnull().sum()

ra               0
dec              0
starthjd         0
endhjd           0
bmag        134974
berr        134974
vmag         11571
verr         11571
rmag        134974
rerr        134974
imag          7495
ierr          7495
npts             0
dtype: int64

columns_to_remove


In [17]:
columns_to_remove = ['bmag','berr','rmag','rerr']
df[numeric_columns].isnull().sum()

ra               0
dec              0
starthjd         0
endhjd           0
bmag        134974
berr        134974
vmag         11571
verr         11571
rmag        134974
rerr        134974
imag          7495
ierr          7495
npts             0
dtype: int64

In [18]:
df_numeric = df[numeric_columns].drop(columns=columns_to_remove)

In [19]:
df_numeric.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ra,dec,starthjd,endhjd,vmag,verr,imag,ierr,npts
0,103.0431,0.4158,2453047.76,2453058.94,NaN,NaN,NaN,NaN,167
1,102.9154,0.4254,2453047.76,2453058.94,NaN,NaN,NaN,NaN,167
2,102.9629,0.4276,2453047.76,2453058.94,NaN,NaN,NaN,NaN,167
3,103.0485,0.4260,2453047.76,2453058.94,NaN,NaN,NaN,NaN,167
4,102.9760,0.4326,2453047.76,2453058.94,NaN,NaN,NaN,NaN,167


In [20]:
df_numeric.isnull().sum()

ra              0
dec             0
starthjd        0
endhjd          0
vmag        11571
verr        11571
imag         7495
ierr         7495
npts            0
dtype: int64

In [21]:
df_numeric = df_numeric.dropna()

In [22]:
df_numeric.shape

(123830, 9)

In [23]:
df_numeric.isna().sum()

ra          0
dec         0
starthjd    0
endhjd      0
vmag        0
verr        0
imag        0
ierr        0
npts        0
dtype: int64

In [24]:
df_numeric

,ra,dec,starthjd,endhjd,vmag,verr,imag,ierr,npts
3961,154.1162,-46.5045,2450246.45,2450939.64,18.251,0.014,17.462,0.017,105
3962,154.1171,-46.4989,2450246.45,2450940.53,20.216,0.033,18.386,0.022,113
3963,154.1172,-46.4909,2450246.45,2450940.53,20.623,0.038,19.206,0.032,110
3965,154.1176,-46.4801,2450246.45,2450940.53,19.365,0.018,18.226,0.022,109
3966,154.1179,-46.4671,2450246.45,2450939.57,19.535,0.018,18.653,0.025,108
...,...,...,...,...,...,...,...,...,...
138930,254.1711,-4.0903,2450566.77,2451318.86,18.904,0.016,18.022,0.014,49
138931,254.1712,-4.0854,2450931.78,2451052.53,16.355,0.010,15.232,0.010,14
138932,254.1712,-4.2396,2450566.77,2451318.86,18.751,0.016,17.712,0.017,45
138933,254.1712,-4.0354,2450566.77,2451318.86,20.865,0.054,19.714,0.044,51
